In [1]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score, roc_curve


In [4]:
#get threshold of TNFalpha and CD107a
threshold = pd.read_csv('./marker_threshold.csv', sep=',')
for c in threshold.columns:
    threshold = threshold.rename(columns={c: c.split("_")[0]})
thres1 = threshold.loc[0, 'TNFa']
thres2 = threshold.loc[0, 'CD107a']
print(thres1, thres2)

1.451619591 1.192596599


In [5]:
data = {}
data['hr_start']      = []
data['hr_end']        = []
data['ratio']         = []
data['size']          = []
data['type']          = []
data['mean_accuracy'] = []
data['std']           = []
data['roc-auc']       = []
mean_fpr = np.linspace(0, 1, 100)
hr_start = 5
hr_end   = 8
o_ratio  = 6

In [6]:
# Load dataset
for num, time in enumerate(range(hr_start, hr_end + 1)):
    if not num : 
        df_0 = pd.read_csv(f'NK_Cell/transformedtxt/IL2/{time}hr_IL2_panel1.fcs_spill_applied_Ungated.txt', sep='\t', skiprows=1, index_col=False)
        dfs  = [df_0]
    else:
        new_df_0 = pd.read_csv(f'NK_Cell/transformedtxt/IL2/{time}hr_IL2_panel1.fcs_spill_applied_Ungated.txt', sep='\t', skiprows=1, index_col=False)
        dfs.append(new_df_0)
df_0 = pd.concat(dfs).reset_index(drop=True)    
df_0 = df_0.drop("Event #", axis=1)
for c in df_0.columns:
    df_0 = df_0.rename(columns={c: c.split("_")[0]})
   

In [ ]:
 
# impute negative as 0 
df_0[df_0 < 0] = 0
# drop unused responses
targets = ['IFNg', 'IL3', 'MIP1a', 'GMCSF', 'MIP1b']
df_0    = df_0.drop(targets, axis=1)
c_list  = ['CD57', 'CD2', 'NKG2A', 'CD16', 'CD94', 'CD45RA', 'NKp30', 'CRACC', 'CRTAM', 'CD56', 'CD45', 'DNAM1', 'NKp46', 'CD161', 'CD8', 'NKG2C', 'CD96', 'LILRB1', 'KLRG1', 'DAP12', 'SAP', 'LAIR1', 'CD7', 'TIGIT', 'CD69', 'NKG2D', 'CD11a', 'EAT2', 'NTB', '2B4']

In [ ]:
# Split data by ratio
df_0_  = df_0.assign(odds=0)
df_0_['odds'] = (df_0_['TNFa']/thres1+0.0001)/(df_0_['CD107a']/thres2+0.0001)
E  = df_0_[((df_0_['odds'] >= o_ratio) & (df_0_['TNFa'] >= thres1))].dropna(axis=1).drop(['odds'], axis=1)
F  = df_0_[(df_0_['odds'] <= 1/o_ratio) & (df_0_['CD107a'] >= thres2)].dropna(axis=1).drop(['odds'], axis=1)
E  = E.assign(label = 'TNFa')
F  = F.assign(label = 'CD107a')
EF = pd.concat([E, F])  
EF = shuffle(EF)

In [ ]:
# construct training set
dropped  = ['CD107a', 'TNFa', 'label']
features = EF.drop(dropped, axis=1).columns
N02 = EF.index
X02 = EF.drop(dropped, axis=1).values
y02 = EF['label'].values.copy()
y02[y02 == 'TNFa']   = 0
y02[y02 == 'CD107a'] = 1
y02 = y02.astype(int)

In [ ]:
# ROC curve setup
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.xticks(rotation=0, fontsize=20)
plt.yticks(fontsize=20)

In [7]:
# XGboost
print("- XGboost -")
feature_names = [x for x in df_0.columns if x not in dropped + ['label', 'label_response', 'Event #']]
kf = KFold(n_splits = 5)
for (num, (X, y)) in enumerate(zip([X02], [y02])):
    print("dataset : ", num, len(X))
    acc          = []
    dm_acc       = []
    all_tpr      = []
    all_auroc    = []
    max_accuracy = 0
    for (f, (train_index, test_index)) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        dtrain = xgb.DMatrix(data = X_train, label = y_train, feature_names = feature_names)
        dtest  = xgb.DMatrix(data = X_test, label = y_test, feature_names = feature_names)
        param  = {'objective': 'binary:logistic', 'nthread': 1, 'eval_metric': ['error', 'auc'], 'booster': 'gbtree'}
        # train XGBoost
        clf    = xgb.train(param, dtrain, verbose_eval=False, early_stopping_rounds=2, evals=[(dtest, 'test')])
        y_test = dtest.get_label()
        preds  = clf.predict(dtest)
        predictions   = [round(value) for value in preds]
        test_accuracy = accuracy_score(y_test, predictions).round(3)
        print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))
        acc.append(test_accuracy)
        if accuracy >= max_accuracy:
            max_accuracy = accuracy
        # plot XGboost importance
        ax  = xgb.plot_importance(clf, importance_type='gain')
        fig = ax.figure
        fig.set_size_inches(10, 8)
        ax.figure.tight_layout()
        ax.figure.savefig(f'XGB_importance.png', dpi=300)
        # save iteration ROC
        fpr, tpr, _ = roc_curve(y_test, predictions)
        all_tpr.append(np.interp(mean_fpr, fpr, tpr))
        auroc  = roc_auc_score(y_test, predictions)
        all_auroc.append(auroc)
    # caculate mean ROC
    mean_tpr = np.mean(all_tpr, axis=0)
    plt.plot(mean_fpr, mean_tpr, label=f'XGboost (AUROC={np.mean(all_auroc)})')
    data['hr_start'].append(hr_start)
    data['hr_end'].append(hr_end)
    data['ratio'].append(o_ratio)
    data['size'].append(len(EF))
    data['type'].append("XGB")
    data['mean_accuracy'].append(np.mean(acc))
    data['std'].append(np.std(acc))
    data['roc-auc'].append(np.mean(all_auroc))
    print(f"MEAN ACC : {np.mean(acc)}")
    print(f"STD : {np.std(acc)}")
    print(f"MEAN AUROC: {np.mean(all_auroc)}")
    # plot ROCs
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title(f'ROC curve')
    plt.legend(loc='best', fontsize='xx-large')
    plt.savefig(f"*Normalized/*ROC_{hr_start}_{hr_end}_{o_ratio}.png", dpi=300)
    plt.close('all')